In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from find_screen_utils import *
from __future__ import division

#Add Gaze Position to Video

In [4]:
gaze = pd.read_csv('images/tobii/border/video/video_disp.csv')
vid_sync = pd.read_csv('images/tobii/border/video/video_disp_video_sync.csv')

In [5]:
vid_sync.head()

,Unnamed: 0,VTS time,VTS validity
0,69017560,0,0
1,69617620,599844,0


In [6]:
gaze['Vid_Time'] = (gaze['index'] - 69017560)/1000000

In [7]:
gaze = gaze[gaze.Vid_Time > 0] # only start tracking eyes once video starts

In [11]:
infile = 'images/tobii/border/video/fullstream.mp4'
outfile = 'images/tobii/border/video/video_disp_gaze.m4v'

In [12]:
vid = cv2.VideoCapture(infile)

size = (1920, 1080)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS)*2, size, True)
# note doubled framerate to include all eye tracking coordinates

i = 0
gaze_val = gaze['gaze position validity'].values
gaze_x = gaze['gaze position x'].values
gaze_y = gaze['gaze position y'].values

while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        for _ in range(2):
            frame_cp = frame.copy()
            if gaze_val[i] == 0:
                gp_x = gaze_x[i]
                gp_y = gaze_y[i]
                cv2.circle(frame_cp, (int(1920*gp_x), int(1080*gp_y)), 20, [255,0,0], 2)
            out.write(frame_cp)
            i += 1
    else:
        break


vid.release()
out.release()

#Extract Screen From Gaze Video

In [2]:
infile = 'images/tobii/border/video/video_disp_gaze.m4v'
outfile = 'images/tobii/border/video/video_disp_gaze_screen2.m4v'

In [3]:
vid = cv2.VideoCapture(infile)

size = (1280, 720)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS), size, True)

last_good = np.zeros((720, 1280, 3), dtype=np.uint8)  # represents the last good frame

i = 1
tot = vid.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        proc_frame = process_frame_lines(frame, 'light')
        
        if proc_frame is None:  # if screen isn't found, save image of frame
            out.write(last_good)  # write the last good frame to the video
            cv2.imwrite('images/tobii/border/video/dropped/frame%i.bmp'%(i), frame)
            pass
        else:  # frame is good
            last_good = proc_frame
            out.write(proc_frame)
        i += 1
        if i%50 == 0:
            print '%0.2f%%'%((i/tot)*100)
    else:
        break


vid.release()
out.release()

4.16%
8.31%
12.47%
16.63%
20.78%
24.94%
29.09%
33.25%
37.41%
41.56%
45.72%
49.88%
54.03%
58.19%
62.34%
66.50%
70.66%
74.81%
78.97%
83.13%
87.28%
91.44%
95.59%
99.75%
